In [21]:
!pip install vncorenlp
!pip install transformers torch


ERROR: Operation cancelled by user


# ***`LIBRARIES`***

In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# ***`DOWNLOAD PhoBERT Base`***

In [23]:
from transformers import AutoTokenizer, AutoModel

# Tải tokenizer
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

# Tải mô hình pretrained PhoBERT (encoder)
model = AutoModel.from_pretrained("vinai/phobert-base")


# ***`READ DATA`***

In [24]:
df = pd.read_excel('TopicModeling_Final.xlsx', engine='openpyxl') # For xlsx files
df

,Category,Content
0,Văn hóa & lối sống,Bái Đính cổ tự là ngôi chùa được xây dựng trên...
1,Kinh doanh & quản trị,"Vận động viên của Singapore có thể là luật sư,..."
2,Y tế & sức khỏe,"Đầu năm học mới, tôi cân thử chiếc ba lô của c..."
3,Chính trị & chính sách,“Tâm tư của tôi cái gì luật không cấm thì phải...
4,Văn hóa & lối sống,"Mẹ về nhà, lưng áo đẫm mồ hôi, ngồi đếm những ..."
...,...,...
3328,Môi trường,Chúng ta đang rửa tay theo cách rất khác với m...
3329,Kinh doanh & quản trị,"Tôi gặp Chokchai Koisrichai - giám đốc, nhà đi..."
3330,Kinh doanh & quản trị,Suốt bao năm tôi ở nhà thuê vì không biết sau ...
3331,Giáo dục & tri thức,Tôi vừa đi khảo sát một số trường học ở một hu...


In [25]:
# Bước 1: Chuyển chữ thường, strip và chuẩn hóa unicode
import unicodedata

def clean_label(label):
    label = unicodedata.normalize('NFC', label)
    label = label.lower().strip()
    label = label.replace("  ", " ")  # nếu có double space
    return label

df['Category_clean'] = df['Category'].apply(clean_label)

# Bước 2: Mapping về nhãn chuẩn (tên đẹp)
category_mapping = {
    'văn hóa & lối sống': 'Văn hóa & lối sống',
    'chính trị & chính sách': 'Chính trị & chính sách',
    'kinh doanh & quản trị': 'Kinh doanh & quản trị',
    'giáo dục & tri thức': 'Giáo dục & tri thức',
    'y tế & sức khỏe': 'Y tế & sức khỏe',
    'môi trường': 'Môi trường'
    }

# Bước 3: Gán lại nhãn cuối
df['Category_standard'] = df['Category_clean'].map(category_mapping)
df['Category_standard'].value_counts()

df = df.drop(columns=['Category_clean', 'Category'])

# ***`Bootstrap Oversampling`***

In [26]:
from sklearn.utils import resample

max_count = df['Category_standard'].value_counts().max()
balanced_dfs = []

for label in df['Category_standard'].unique():
    group = df[df['Category_standard'] == label]
    resampled = resample(group, replace=True, n_samples=max_count, random_state=42)
    balanced_dfs.append(resampled)

df = pd.concat(balanced_dfs).sample(frac=1, random_state=42).reset_index(drop=True)


# ***`DATA PREPROCESSING`***

In [27]:
import unicodedata
import re
import string
import pandas as pd
from transformers import AutoTokenizer

# Load PhoBERT tokenizer
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

# Load stopwords từ file "vietnamese-stopwords.txt"
def load_stopwords(filepath):
    with open(filepath, encoding='utf-8') as f:
        stopwords = set(line.strip() for line in f if line.strip())
    return stopwords

stopwords = load_stopwords("vietnamese-stopwords.txt")

# 1. Chuẩn hóa unicode NFC
def normalize_unicode(text):
    return unicodedata.normalize('NFC', text)

# 2. Làm sạch văn bản: viết thường, bỏ số, dấu câu, khoảng trắng thừa
def text_normalizer(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = re.sub(f"[{re.escape(string.punctuation)}]", " ", text)
    text = re.sub("\s+", " ", text).strip()
    return text

# 3. Loại bỏ stopwords
def remove_stopwords(text, stopwords):
    tokens = text.split()
    filtered = [word for word in tokens if word not in stopwords]
    return " ".join(filtered)

# 4. Tiền xử lý văn bản thành văn bản sạch (không mã hóa)
def clean_text_for_phobert(text, stopwords):
    text = normalize_unicode(text)
    text = text_normalizer(text)
    text = remove_stopwords(text, stopwords)
    return text

# 5. Tokenize thành input_ids
def encode_text(text, max_len=256):
    return tokenizer.encode(
        text,
        max_length=max_len,
        truncation=True,
        padding='max_length'
    )

# --- Áp dụng lên DataFrame ---
# df là DataFrame đã có cột 'Content'

# Tạo cột văn bản đã xử lý
df['Content_cleaned'] = df['Content'].apply(lambda x: clean_text_for_phobert(x, stopwords))

# Tạo cột input_ids cho PhoBERT
df['input_ids'] = df['Content_cleaned'].apply(lambda x: encode_text(x))


KeyboardInterrupt: 

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df['label'] = le.fit_transform(df['Category_standard'])
num_labels = len(le.classes_)

In [ ]:
df.to_excel('df_final.xlsx')

# ***`Train/Test Split`***

In [28]:
df_final = pd.read_excel('df_final.xlsx')
df_final = df_final.drop(columns=['Unnamed: 0', 'Content', 'Category_standard', 'Content_cleaned'])
df_final

,input_ids,label
0,"[0, 441, 9866, 1560, 1294, 4721, 1103, 2935, 2...",2
1,"[0, 2404, 6928, 2487, 229, 1824, 3078, 7564, 6...",2
2,"[0, 328, 2201, 328, 9667, 2710, 1713, 12053, 8...",2
3,"[0, 61610, 10893, 49592, 1701, 564, 61610, 353...",0
4,"[0, 13397, 409, 109, 441, 1171, 2497, 2194, 61...",2
...,...,...
4759,"[0, 4368, 1430, 1895, 14294, 418, 119, 289, 74...",1
4760,"[0, 238, 222, 4698, 1746, 401, 940, 4698, 1031...",4
4761,"[0, 2925, 2792, 1197, 9645, 1080, 30768, 2183,...",0
4762,"[0, 286, 9393, 2288, 18116, 176, 853, 2615, 85...",3


In [29]:
from sklearn.model_selection import train_test_split
# === Chia tập dữ liệu train - val - test
train_val_df, test_df = train_test_split(df_final, test_size=0.15, random_state=42, stratify=df_final['label'])
train_df, val_df = train_test_split(train_val_df, test_size=0.15, random_state=42, stratify=train_val_df['label'])


# ***`Torch Format Dataset`***

In [30]:
# import torch
# from torch.utils.data import Dataset

# class PhoBertDataset(Dataset):
#     def __init__(self, data):
#         self.input_ids = data['input_ids'].tolist()
#         self.labels = data['label'].tolist()

#     def __getitem__(self, idx):
#         return {
#             'input_ids': torch.tensor(self.input_ids[idx], dtype=torch.long),
#             'attention_mask': torch.tensor([1 if id != 1 else 0 for id in self.input_ids[idx]], dtype=torch.long),
#             'labels': torch.tensor(self.labels[idx], dtype=torch.long)
#         }

#     def __len__(self):
#         return len(self.labels)


# # === Khởi tạo datasets
# train_dataset = PhoBertDataset(train_df)
# val_dataset = PhoBertDataset(val_df)
# test_dataset = PhoBertDataset(test_df)


In [31]:
import torch
from torch.utils.data import Dataset
import ast  # Import ast to evaluate string representation of lists

class PhoBertDataset(Dataset):
    def __init__(self, data):
        # Convert strings to lists using ast.literal_eval
        self.input_ids = data['input_ids'].apply(ast.literal_eval).tolist()
        self.labels = data['label'].tolist()

    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.input_ids[idx], dtype=torch.long),
            'attention_mask': torch.tensor([1 if id != 1 else 0 for id in self.input_ids[idx]], dtype=torch.long),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }

    def __len__(self):
        return len(self.labels)


# === Khởi tạo datasets
train_dataset = PhoBertDataset(train_df)
val_dataset = PhoBertDataset(val_df)
test_dataset = PhoBertDataset(test_df)

# ***`PhoBERT Classification`***

In [32]:
import numpy as np
import torch
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from transformers import Trainer

# === Cấu hình số lượng nhãn
num_labels = df_final['label'].nunique()

# === Tải mô hình PhoBERT
model = AutoModelForSequenceClassification.from_pretrained(
    "vinai/phobert-base",
    num_labels=num_labels
)

# === Tính class weights từ tập train
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_df['label']),
    y=train_df['label']
)
class_weights = torch.tensor(class_weights, dtype=torch.float)

# === Cấu hình training
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=6,
    learning_rate=2e-5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    greater_is_better=True,
    logging_dir="./logs",
    report_to="none"
)

# === Hàm tính metric
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    labels = p.label_ids
    return {
        "accuracy": accuracy_score(labels, preds),
        "precision_macro": precision_score(labels, preds, average='macro'),
        "recall_macro": recall_score(labels, preds, average='macro'),
        "f1_macro": f1_score(labels, preds, average='macro'),
    }

# === Trainer có gán class weights
class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):  # <- thêm **kwargs
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights.to(model.device))
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss

# === Huấn luyện mô hình
trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

trainer.train()


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision Macro,Recall Macro,F1 Macro
1,1.139600,0.869620,0.722039,0.722178,0.722530,0.720043
2,0.694400,0.690583,0.776316,0.779719,0.776775,0.771399
3,0.477800,0.734882,0.804276,0.809197,0.804650,0.805246
4,0.372600,0.678841,0.848684,0.847079,0.849026,0.847517
5,0.270400,0.723581,0.850329,0.852124,0.850547,0.848756
6,0.215800,0.709413,0.851974,0.850398,0.852246,0.850526


TrainOutput(global_step=2586, training_loss=0.528444707624136, metrics={'train_runtime': 1186.7178, 'train_samples_per_second': 17.398, 'train_steps_per_second': 2.179, 'total_flos': 2716192971380736.0, 'train_loss': 0.528444707624136, 'epoch': 6.0})

In [33]:
# === Đánh giá mô hình trên tập test
test_metrics = trainer.evaluate(test_dataset)
print("Test set evaluation:")
for k, v in test_metrics.items():
    print(f"{k}: {v:.4f}")


Test set evaluation:
eval_loss: 0.6766
eval_accuracy: 0.8615
eval_precision_macro: 0.8619
eval_recall_macro: 0.8614
eval_f1_macro: 0.8613
eval_runtime: 10.7971
eval_samples_per_second: 66.2220
eval_steps_per_second: 8.3360
epoch: 6.0000


In [34]:
preds_output = trainer.predict(test_dataset)
pred_labels = np.argmax(preds_output.predictions, axis=1)

# Nếu muốn lưu để phân tích sau:
test_df["pred_label"] = pred_labels
test_df.to_excel("test_predictions.xlsx", index=False)
